importing required packages and models

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import metrics
# importing models
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from xgboost import XGBClassifier

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split # Import train_test_split function

from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import seaborn as sns
# import warnings filter
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

loading the descriptor datasets

In [2]:
train=pd.read_csv("desc_train_I.csv")
test=pd.read_csv("desc_test_I.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75377 entries, 0 to 75376
Columns: 214 entries, MaxEStateIndex to Mol
dtypes: float64(104), int64(107), object(3)
memory usage: 123.1+ MB


Checking for the null values and filling those nulls with mean

In [4]:
null=train.isnull().sum()
null=null[null>0]
null
train =train.fillna(train.mean())
test=test.fillna(train.mean())

In [5]:
train.dtypes

MaxEStateIndex       float64
MinEStateIndex       float64
MaxAbsEStateIndex    float64
MinAbsEStateIndex    float64
qed                  float64
                      ...   
Id                    object
Expected               int64
Compound              object
Assay_Id               int64
Mol                   object
Length: 214, dtype: object

Removing the high correlation columns

In [6]:
corr= train.corr(method='spearman')
high_corr_cols = []
for i in range(len(corr.columns)):
    for j in range(i):
        if abs(corr.iloc[i, j]) > 0.8:
            high_corr_cols.append((corr.columns[i], corr.columns[j]))
print("High correlation pairs:")
for pair in high_corr_cols:
    print(pair)

High correlation pairs:
('MaxAbsEStateIndex', 'MaxEStateIndex')
('HeavyAtomMolWt', 'MolWt')
('ExactMolWt', 'MolWt')
('ExactMolWt', 'HeavyAtomMolWt')
('NumValenceElectrons', 'MolWt')
('NumValenceElectrons', 'HeavyAtomMolWt')
('NumValenceElectrons', 'ExactMolWt')
('MaxAbsPartialCharge', 'MinPartialCharge')
('MinAbsPartialCharge', 'MaxPartialCharge')
('FpDensityMorgan2', 'FpDensityMorgan1')
('FpDensityMorgan3', 'FpDensityMorgan2')
('BertzCT', 'MolWt')
('BertzCT', 'HeavyAtomMolWt')
('BertzCT', 'ExactMolWt')
('BertzCT', 'NumValenceElectrons')
('Chi0', 'MolWt')
('Chi0', 'HeavyAtomMolWt')
('Chi0', 'ExactMolWt')
('Chi0', 'NumValenceElectrons')
('Chi0', 'BertzCT')
('Chi0n', 'MolWt')
('Chi0n', 'HeavyAtomMolWt')
('Chi0n', 'ExactMolWt')
('Chi0n', 'NumValenceElectrons')
('Chi0n', 'BertzCT')
('Chi0n', 'Chi0')
('Chi0v', 'MolWt')
('Chi0v', 'HeavyAtomMolWt')
('Chi0v', 'ExactMolWt')
('Chi0v', 'NumValenceElectrons')
('Chi0v', 'BertzCT')
('Chi0v', 'Chi0')
('Chi0v', 'Chi0n')
('Chi1', 'MolWt')
('Chi1', 'Hea

In [7]:
first_items = [pair[0] for pair in high_corr_cols ]
second_items = [pair[1] for pair in high_corr_cols ]
list1=list(set(first_items))
list2=list(set(second_items))
list1.append("MinPartialCharge")

Dropping the correlated features from train file

In [8]:
train.drop(list1,axis=1,inplace=True)

train test split

In [9]:
X=train.drop(["Expected","Id","Compound","Mol"],axis=1)
y=train["Expected"]

In [10]:
X

,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,NumRadicalElectrons,MaxPartialCharge,FpDensityMorgan1,BCUT2D_MWHI,BCUT2D_MWLOW,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Assay_Id2
0,9.316200,-1.533785,0.150485,0.794714,317.599,0,0.200859,0.684211,35.582517,9.922283,...,0,0,0,0,0,0,0,0,0,1644
1,10.532611,0.333788,0.333788,0.516641,156.269,0,0.129340,1.000000,16.137172,10.087958,...,0,0,0,0,0,0,0,4,0,2451
2,2.433032,0.000000,0.000000,0.251327,362.086,0,0.078199,0.416667,35.453001,10.070986,...,0,0,0,0,0,0,0,12,0,1384
3,10.355080,-0.613825,0.282361,0.487998,255.665,0,0.256798,1.588235,35.495696,10.251808,...,0,0,0,0,0,0,0,0,0,16
4,0.000000,0.000000,0.000000,0.237972,149.894,0,1.000000,1.000000,26.919799,10.157167,...,0,0,0,0,0,0,0,0,0,1856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,11.460021,-3.868472,0.053611,0.712426,230.245,0,0.420384,1.333333,32.233271,10.319174,...,1,0,0,0,0,0,0,0,0,33
75373,5.928972,-2.841623,0.082346,0.720533,313.747,0,0.382438,1.333333,35.495702,10.415690,...,0,0,0,0,0,0,0,0,0,1632
75374,4.975926,0.848333,0.848333,0.596343,167.258,0,0.158935,1.300000,32.148937,10.336508,...,0,0,0,0,1,0,0,0,0,1373
75375,10.241948,0.324028,0.324028,0.519485,128.215,0,0.122557,1.444444,16.129374,10.015813,...,0,0,0,0,0,0,0,0,0,2


Forward Feature Selection using Decision Tree Classifier

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset and split it into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an empty list to keep track of selected features
selected_features = []

# Define a function to calculate accuracy of a model
def calculate_accuracy(X_train, X_test, y_train, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Start with an empty set of features and iteratively add the best feature
while len(selected_features) < len(X_train.columns):
    remaining_features = list(set(X_train.columns) - set(selected_features))
    best_accuracy = 0
    best_feature = ''
    
    # Loop through remaining features and find the one that improves the accuracy the most
    for feature in remaining_features:
        features_to_use = selected_features + [feature]
        X_train_subset = X_train[features_to_use]
        X_test_subset = X_test[features_to_use]
        accuracy = calculate_accuracy(X_train_subset, X_test_subset, y_train, y_test)
        
        # If this feature improves the accuracy, update the best feature
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_feature = feature
    
    # Add the best feature to the list of selected features
    selected_features.append(best_feature)
    print("Selected Features: ", selected_features)
    print("Accuracy: ", best_accuracy)


Selected Features:  ['Assay_Id2']
Accuracy:  0.8723136110374105
Selected Features:  ['Assay_Id2', 'fr_halogen']
Accuracy:  0.8737728840541258
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2']
Accuracy:  0.8739055452374636
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime']
Accuracy:  0.8743698593791457
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan']
Accuracy:  0.8746351817458212
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8']
Accuracy:  0.8746351817458212
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide']
Accuracy:  0.8746351817458212
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH']
Accuracy:  0.8746351817458212
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylati

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine']
Accuracy:  0.8738392146457946
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_am

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate']
Accuracy:  0.8699920403289997
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid']
Accuracy:  0.8660785354205359
Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid', 'fr_imidazole', 'NumAliphaticCarbocycles', 'fr_Ar_OH', 'VSA_EState3', 'PEOE_VSA12', 'MaxPartialCharge', 'SMR_VSA10', 'MolLogP', 'fr_aryl_methyl']
Accurac

...

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid', 'fr_imidazole', 'NumAliphaticCarbocycles', 'fr_Ar_OH', 'VSA_EState3', 'PEOE_VSA12', 'MaxPartialCharge', 'SMR_VSA10', 'MolLogP', 'fr_aryl_methyl', 'VSA_ES

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid', 'fr_imidazole', 'NumAliphaticCarbocycles', 'fr_Ar_OH', 'VSA_EState3', 'PEOE_VSA12', 'MaxPartialCharge', 'SMR_VSA10', 'MolLogP', 'fr_aryl_methyl', 'VSA_ES

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid', 'fr_imidazole', 'NumAliphaticCarbocycles', 'fr_Ar_OH', 'VSA_EState3', 'PEOE_VSA12', 'MaxPartialCharge', 'SMR_VSA10', 'MolLogP', 'fr_aryl_methyl', 'VSA_ES

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid', 'fr_imidazole', 'NumAliphaticCarbocycles', 'fr_Ar_OH', 'VSA_EState3', 'PEOE_VSA12', 'MaxPartialCharge', 'SMR_VSA10', 'MolLogP', 'fr_aryl_methyl', 'VSA_ES

Selected Features:  ['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'SMR_VSA8', 'fr_azide', 'fr_SH', 'fr_prisulfonamd', 'fr_dihydropyridine', 'fr_diazo', 'fr_isocyan', 'fr_guanido', 'fr_barbitur', 'fr_benzodiazepine', 'fr_lactam', 'SlogP_VSA9', 'fr_Ar_COO', 'EState_VSA11', 'fr_ArN', 'fr_HOCCN', 'fr_nitro_arom_nonortho', 'fr_nitroso', 'fr_thiocyan', 'fr_tetrazole', 'fr_C_S', 'NumRadicalElectrons', 'fr_aldehyde', 'fr_hdrzine', 'fr_thiazole', 'fr_amidine', 'fr_piperzine', 'fr_priamide', 'fr_N_O', 'fr_hdrzone', 'fr_thiophene', 'fr_azo', 'fr_term_acetylene', 'fr_morpholine', 'fr_furan', 'fr_quatN', 'fr_epoxide', 'fr_oxazole', 'fr_Ar_NH', 'fr_alkyl_carbamate', 'fr_Ndealkylation1', 'fr_Al_COO', 'fr_Imine', 'fr_nitro', 'fr_Al_OH', 'fr_sulfone', 'fr_sulfonamd', 'fr_ketone', 'fr_NH2', 'fr_lactone', 'fr_phos_acid', 'fr_imidazole', 'NumAliphaticCarbocycles', 'fr_Ar_OH', 'VSA_EState3', 'PEOE_VSA12', 'MaxPartialCharge', 'SMR_VSA10', 'MolLogP', 'fr_aryl_methyl', 'VSA_ES

Selected features

In [12]:
selected_features=['Assay_Id2', 'fr_halogen', 'fr_Ndealkylation2', 'fr_oxime', 'fr_isothiocyan', 'fr_prisulfonamd', 
                   'fr_isocyan', 'fr_barbitur', 'fr_dihydropyridine', 'SlogP_VSA9', 'fr_SH', 'fr_diazo',
                   'fr_guanido', 'SMR_VSA8', 'fr_benzodiazepine', 'fr_azide', 'fr_nitroso', 'fr_thiocyan', 
                   'fr_lactam', 'EState_VSA11', 'fr_aldehyde', 'fr_Ar_COO', 'fr_ArN', 'fr_nitro_arom_nonortho', 
                   'fr_HOCCN', 'fr_C_S', 'fr_priamide', 'fr_tetrazole', 'fr_amidine', 'fr_thiazole', 'fr_azo',
                   'NumRadicalElectrons', 'fr_thiophene', 'fr_hdrzone', 'fr_furan', 'fr_morpholine',
                   'fr_term_acetylene', 'fr_hdrzine', 'fr_epoxide', 'fr_oxazole', 'fr_piperzine', 'fr_Ar_NH', 
                   'fr_alkyl_carbamate', 'fr_N_O', 'fr_NH2', 'fr_sulfonamd', 'fr_Al_COO', 'fr_quatN', 'fr_nitro',
                   'fr_Imine', 'fr_Al_OH', 'fr_Ndealkylation1', 'fr_piperdine', 'fr_lactone', 'fr_allylic_oxid', 
                   'fr_sulfone', 'fr_alkyl_halide', 'fr_phos_acid', 'fr_ester', 'BCUT2D_MRLOW', 'VSA_EState5', 
                   'PEOE_VSA3', 'SMR_VSA3', 'PEOE_VSA1', 'PEOE_VSA9', 'PEOE_VSA11', 'fr_para_hydroxylation',
                   'fr_pyridine', 'VSA_EState2', 'SlogP_VSA4', 'NumAliphaticCarbocycles', 'SlogP_VSA1', 'fr_C_O', 
                   'fr_NH0', 'NumAromaticHeterocycles', 'PEOE_VSA12', 'NumSaturatedHeterocycles', 'fr_urea',
                   'fr_aniline', 'fr_sulfide', 'fr_ether', 'fr_aryl_methyl', 'SlogP_VSA10', 'fr_amide', 'NHOHCount', 
                   'PEOE_VSA4', 'fr_NH1', 'fr_imide', 'fr_methoxy', 'MinEStateIndex', 'fr_ketone', 'PEOE_VSA6', 
                   'qed', 'SlogP_VSA7', 'FractionCSP3', 'NumAliphaticHeterocycles', 'fr_imidazole', 'SMR_VSA4', 
                   'BCUT2D_MWHI', 'SMR_VSA1', 'PEOE_VSA13', 'PEOE_VSA5', 'MaxEStateIndex', 'VSA_EState7', 
                   'EState_VSA10', 'VSA_EState1', 'MolWt', 'SMR_VSA10', 'EState_VSA6', 'PEOE_VSA10', 
                   'NumAliphaticRings', 'EState_VSA9', 'VSA_EState3', 'EState_VSA2', 'VSA_EState8', 'TPSA', 
                   'SMR_VSA2', 'SMR_VSA9']

In [13]:
X=train[selected_features]
y=train["Expected"]

In [14]:
X

,Assay_Id2,fr_halogen,fr_Ndealkylation2,fr_oxime,fr_isothiocyan,fr_prisulfonamd,fr_isocyan,fr_barbitur,fr_dihydropyridine,SlogP_VSA9,...,EState_VSA6,PEOE_VSA10,NumAliphaticRings,EState_VSA9,VSA_EState3,EState_VSA2,VSA_EState8,TPSA,SMR_VSA2,SMR_VSA9
0,1644,3,0,0,0,0,0,0,0,0.0,...,48.530937,11.499024,0,34.802820,18.632400,11.499024,0.000000,40.46,0.0,11.499024
1,2451,0,0,0,0,0,0,0,0,0.0,...,6.923737,5.783245,0,0.000000,0.000000,0.000000,3.890488,17.07,0.0,0.000000
2,1384,1,0,0,0,0,0,0,0,0.0,...,0.000000,0.000000,0,0.000000,0.000000,12.407033,7.356997,0.00,0.0,0.000000
3,16,1,0,0,0,0,0,0,0,0.0,...,17.163120,5.153110,1,11.600940,10.164847,5.959555,1.722057,83.66,0.0,0.000000
4,1856,1,0,0,0,0,0,0,0,0.0,...,0.000000,0.000000,0,0.000000,0.000000,53.533926,0.000000,0.00,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,33,0,0,0,0,0,0,0,0,0.0,...,4.722095,0.000000,0,5.733667,0.000000,4.895483,0.000000,98.49,0.0,0.000000
75373,1632,1,0,0,0,0,0,0,0,0.0,...,0.000000,0.000000,0,36.979037,4.356253,17.335892,5.459905,58.40,0.0,6.010465
75374,1373,0,0,0,0,0,0,0,0,0.0,...,11.336786,0.000000,0,12.217873,0.000000,0.000000,0.000000,15.79,0.0,0.000000
75375,2,0,0,0,0,0,0,0,0,0.0,...,0.000000,6.286161,0,0.000000,0.000000,0.000000,4.214533,17.07,0.0,0.000000


Callng CatBoost Model

In [15]:
from catboost import CatBoostClassifier
# Define the CatBoostClassifier model
#model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, l2_leaf_reg=3, loss_function='Logloss')
model = CatBoostClassifier(iterations=25000, learning_rate=0.1)
# Fit the model on the training data
model.fit(X_train, y_train, verbose=False)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))
f1= f1_score(y_test, y_pred)
print(f1)

Accuracy: 91.27%
0.6661593099949263


Dropping the columns from test file and passing it as input to model to predict the Expected value

In [16]:
x=test["x"]
test.drop(list1,axis=1,inplace=True)
test.drop(["x","Compound","Mol"],axis=1,inplace=True)
y_predicted=model.predict(test)

Exporting the csv file

In [17]:
#y_pred=y_pred+1
Predict = pd.DataFrame({ 'Id': x, 'Predicted':y_predicted })
Predict.to_csv("d1.csv", index=False)